In [52]:
import pandas as pd
import numpy as np
import os 
import re

In [53]:
data=pd.read_csv("bbc-text.csv")
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [54]:
cols=data.columns.tolist()
cols=cols[-1:]+cols[:-1]
data=data[cols]

In [55]:
data.head()

,text,category
0,tv future in the hands of viewers with home th...,tech
1,worldcom boss left books alone former worldc...,business
2,tigers wary of farrell gamble leicester say ...,sport
3,yeading face newcastle in fa cup premiership s...,sport
4,ocean s twelve raids box office ocean s twelve...,entertainment


In [56]:
df=pd.read_csv("glove.6B.50d.txt", sep=" ", quoting=3, header=None, index_col=0)
df.T.iloc[0:5,2:4]

,.,of
1,0.15164,0.70853
2,0.30177,0.57088
3,-0.16763,-0.47160
4,0.17684,0.18048
5,0.31719,0.54449


In [68]:
df.shape[0], df.shape[1], df.T.shape[0], df.T.shape[1]

(400000, 50, 50, 400000)

In [57]:
# make a dictionary of glove
glove={key:val.values for key,val in df.T.items()}
# print shape of a vector
print('Shape of vector representation of \'cake\':', len(glove['cake']))

Shape of vector representation of 'cake': 50


In [69]:
glove['cake']

array([ 0.069443 ,  0.76897  , -0.52978  ,  0.084142 ,  1.1945   ,
        0.37776  , -0.34614  ,  0.089619 ,  0.030415 ,  0.14648  ,
       -0.51823  , -0.11561  ,  1.2622   ,  1.0821   ,  0.35248  ,
        0.10658  ,  0.0088652, -0.039198 ,  0.52789  , -1.1781   ,
        0.79956  , -0.49779  ,  1.1323   ,  0.06991  ,  0.0053372,
        0.049828 , -1.0271   ,  1.0131   ,  1.2817   , -0.45057  ,
        1.529    , -0.31407  , -1.4749   ,  1.7196   , -0.29891  ,
        0.20966  , -0.3686   ,  1.333    ,  0.97123  , -0.50428  ,
        0.66221  ,  0.10779  , -0.69917  , -0.62049  ,  0.25486  ,
        0.84102  ,  0.20349  , -1.0346   , -0.074467 , -0.23133  ])

In [58]:
import pickle

with open('glove.6B.50d.pkl', 'wb') as fp:
    # save the pickle file to disk
    pickle.dump(glove, fp)

In [31]:
len(glove)

399998

In [60]:
len(glove.keys())

399998

In [50]:
def generate_inverse_dictionary():
    ids=[x for x in range(len(glove))]
    
    inverse_dict={v:k for k,v in zip(glove.keys(), ids)}
    
    return inverse_dict


print('First 10 entries in the dictionary:') 
list(generate_inverse_dictionary().items())[:10]

First 10 entries in the dictionary:


[(0, 'the'),
 (1, ','),
 (2, '.'),
 (3, 'of'),
 (4, 'to'),
 (5, 'and'),
 (6, 'in'),
 (7, 'a'),
 (8, '"'),
 (9, "'s")]

In [46]:
# find most similar words; given a context word

def most_similar(word_vec, ntop=5):
    inverse_dictionary=generate_inverse_dictionary()
        # fetch glove vectors
    word_vectors=list(glove.values())
    
    # compute cosine_similarity
    cosine_similarity = (np.dot(word_vectors, word_vec)
           / np.linalg.norm(word_vectors, axis=1)
           / np.linalg.norm(word_vec))
    
    # sort the word ids in descending order based on their similarity score
    word_ids = np.argsort(-cosine_similarity)
    
        # return the most similar words with their similarity score

    return [(inverse_dictionary[x], cosine_similarity[x]) for x in word_ids[1:ntop] if x in inverse_dictionary]

most_similar(glove['king'])

[('prince', 0.8236179693335699),
 ('queen', 0.7839043010964116),
 ('ii', 0.7746230030635105),
 ('emperor', 0.7736247624872924)]

In [47]:
# find airplane - fly + drive
result = glove['airplane'] - glove['fly'] + glove['drive']
# find most similar to result
most_similar(result)

[('drive', 0.7521311772461619),
 ('drives', 0.7406034998194793),
 ('vehicle', 0.7290996539480401),
 ('truck', 0.7106720612617872)]

In [49]:
import pickle

with open('glove.6B.50d.pkl', 'rb') as fp:
    # load glove from disk 
    glove = pickle.load(fp)

In [51]:
def get_embedding_glove(tokens):
    embedding=[]
    for token in tokens:
        if token in glove:
            embedding.append(glove[token])
            
        return np.mean(embeddings, axis=0)

In [ ]:
from gensim.utils import simple_preprocess

# preprocess all the articles of the data set
preprocessed_bbc = bbc_news.text.apply(lambda x: simple_preprocess(x))

# create X from glove
X = preprocessed_bbc.apply(lambda x: get_embedding_glove(x))
X = pd.DataFrame(X.tolist())
print('X shape:', X.shape)

In [ ]:
## rest is same as we do in other models